# [실습1] LangChain으로 간단한 LLM 챗봇 만들기 1


## 실습 목표
---
- LangChain을 활용해서 gpt-4o-mini 모델을 사용하는 챗봇을 개발합니다.
- 짧은 Chain을 구성하고, 이를 활용해서 챗봇을 구현합니다.

## 실습 목차
---

1. **ChatOpenAI Agent 생성:** 사용자의 입력에 대한 ChatGPT의 gpt-4o-mini 모델의 답변을 받아오는 Agent를 생성합니다.

2. **챗봇 Chain 구성**: ChatOpenAI Agent를 비롯하여 챗봇 구현에 필요한 Agent들을 엮어서 챗봇 Chain으로 구성합니다.

3. **챗봇 사용**: 여러분이 구성하신 챗봇을 사용해봅니다.

## 실습 개요
---

LangChain의 Chain을 활용해서 gpt-4o-mini 모델을 활용하는 챗봇을 구현하고, Chain을 형성하는 방법을 이해합니다.


## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.chat_models import ChatOpenAI

In [2]:
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""


## 1. ChatOpenAI Agent 생성


gpt-4o-mini 모델을 사용하는 ChatOpenAI Agent를 생성합니다. 
- ChatOpenAI Agent는 사용자의 입력을 Ollama를 통해 로컬에서 구동한 LLM에 전송하고, 그 답변을 반환합니다.
- 본 RAG 과정에서는 LLM으로 ChatOpenAI를 활용할 것입니다.

In [3]:
# 먼저, gpt-4o-mini 모델을 사용하는 ChatOpenAI 객체를 생성합니다.
llm = ChatOpenAI(model="gpt-4o-mini", api_key="")

C:\Users\timeo\AppData\Local\Temp\ipykernel_15216\4169580261.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", api_key="")


Agent를 구성했으니, 이제 Agent를 사용해봅시다.

### 1-1. Runnable interface

LangChain에서 Chain으로 엮을 수 있는 대부분의 구성 요소 (Agent, Tool 등..)는 "Runnable" protocol을 공유합니다.
- 관련 LangChain API 문서: [langchain_core.runnables.base.Runnable — 🦜🔗 LangChain 0.1.16](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)

Runnable protocol을 공유하는 구성 요소는 모두 아래 세 메서드를 가지고 있습니다:
- stream: 구성 요소의 답변을 순차적으로 반환한다 (stream back)
- invoke: 입력된 값으로 chain을 호출하고, 그 결과를 반환한다.
- batch: 입력값 리스트 (batch)로 chain을 호출하고, 그 결과를 반환한다.

예시로, 저희가 방금 사용한 `ChatOpenAI` Class는 "Runnable" 하기 때문에 `invoke` 메서드를 가지고 있습니다.
- invoke() 메서드를 통해 Agent, Chain 등에 데이터를 입력하고, 그 출력을 받아올 수 있습니다.

`invoke` 메서드를 사용해봅시다. 여기서는 "당신은 누구입니까?" 라는 질문을 입력하면 Agent가 OpenAI API를 통해 Mistral 7B 모델의 답변을 받아 출력할 것입니다.

In [4]:
llm.invoke("당신은 누구입니까?")

AIMessage(content='저는 AI 언어 모델인 ChatGPT입니다. 다양한 질문에 답하고, 정보 제공 및 대화를 나누는 데 도움을 드리기 위해 설계되었습니다. 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 14, 'total_tokens': 58, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-c3aa137f-89a1-4736-9f05-16efbd8184bb-0')

단순 텍스트 뿐만 아니라, 시스템, 사람, AI의 답변을 리스트로 정리하여 입력할 수 있습니다. 

여기서는 LangChain의 `SystemMessage`, `HumanMessage` Class를 활용해봅시다.

In [5]:
messages = [
    SystemMessage("당신은 친절한 AI 어시스턴트 입니다."),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)

시스템 프롬프트에 '친절한 AI 어시스턴트' 라는 역할을 명시하였습니다.

이제 gpt-4o-mini 모델이 아까와 같은 질문에 어떻게 답했는지 확인해봅시다.

In [6]:
response

AIMessage(content='안녕하세요! 저는 AI 어시스턴트입니다. 다양한 질문에 답하고, 정보를 제공하며, 여러분의 필요한 도움을 드리기 위해 여기 있습니다. 학습한 데이터를 바탕으로 여러 주제에 대해 대화할 수 있으며, 언제든지 궁금한 점이나 필요한 정보가 있으면 말씀해 주세요!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 31, 'total_tokens': 100, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-5d2a8609-06c6-4b31-95ec-f80a4aea75ad-0')

같은 질문을 했음에도 자신을 소개하는 문구가 조금 달라진 것 을 확인할 수 있습니다.

### [TODO] 다양한 역할을 적용해서 어떻게 답변이 달라지는지 자유롭게 실험해보세요.

In [7]:
role = "국토교통부 직원"
messages = [
    SystemMessage(f"당신은 {role} 입니다."),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)

In [8]:
response

AIMessage(content='안녕하세요! 저는 국토교통부의 직원으로, 교통, 국토 개발, 주택 정책 등 다양한 분야에서 업무를 수행하고 있습니다. 국민의 안전하고 편리한 생활을 위해 정책을 개발하고, 관련 정보를 제공하는 역할을 하고 있습니다. 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 28, 'total_tokens': 105, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-ecc7183b-ce5d-4e56-bbe0-deb80d26d7e3-0')

## 챗봇 Chain 구성

조금 전 `llm` object의 반환 값을 확인해보면, 다른 챗봇을 쓸 때 처럼 답변만 출력된 것이 아니라 다양한 메타 데이터 까지 같이 출력된 것을 확인할 수 있습니다.

저희가 ChatGPT를 쓸 때를 생각해보면, 챗봇에 이걸 그대로 출력하는건 좀 부자연스럽습니다.

이를 방지하기 위해, 답변을 parsing하는 `StrOutputParser`를 실습 2에서 활용해봅시다.